In [60]:
import pandas as pd
from keras.preprocessing.text import Tokenizer

# Import Libraries
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, LSTM, Embedding,Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D,Conv1D, SimpleRNN
from keras.models import Model
from keras.models import Sequential
from keras import initializers, regularizers, constraints,optimizers, layers
from keras.layers import Dense, Input, Flatten, Dropout,BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential


In [61]:
#read file
file_content = pd.read_csv('spam.csv', encoding = "ISO-8859-1")

In [62]:
#check sample content in the email
file_content['v2'][1]

'Ok lar... Joking wif u oni...'

In [63]:
#Import library
from nltk.corpus import stopwords
from nltk import *
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [64]:
# Remove stop words
stop = stopwords.words('english')
file_content['v2'] = file_content['v2'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop))

In [65]:
# Delete unwanted columns
Email_Data = file_content[['v1', 'v2']]

In [66]:
# Rename column names
Email_Data = Email_Data.rename(columns={"v1":"Target", "v2":"Email"})
Email_Data.head()

,Target,Email
0,ham,"Go jurong point, crazy.. Available bugis n gre..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry 2 wkly comp win FA Cup final tkts 2...
3,ham,U dun say early hor... U c already say...
4,ham,"Nah I think goes usf, lives around though"


In [67]:
#Import library
from nltk.corpus import stopwords
from nltk import *
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [68]:
# Remove stop words
stop = stopwords.words('english')
file_content['v2'] = file_content['v2'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop))

In [69]:
# Delete unwanted columns
Email_Data = file_content[['v1', 'v2']]

In [70]:
# Rename column names
Email_Data = Email_Data.rename(columns={"v1":"Target", "v2":"Email"})
Email_Data.head()

,Target,Email
0,ham,"Go jurong point, crazy.. Available bugis n gre..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry 2 wkly comp win FA Cup final tkts 2...
3,ham,U dun say early hor... U c already say...
4,ham,"Nah I think goes usf, lives around though"


In [71]:
#Delete punctuations, convert text in lower case and delete the double space
Email_Data['Email'] = Email_Data['Email'].apply(lambda x:re.sub('[!@#$:).;,?&]','', x.lower()))
Email_Data['Email'] = Email_Data['Email'].apply(lambda x:re.sub(' ', ' ', x))
Email_Data['Email'].head(5)

0    go jurong point crazy available bugis n great ...
1                              ok lar joking wif u oni
2    free entry 2 wkly comp win fa cup final tkts 2...
3                  u dun say early hor u c already say
4             nah i think goes usf lives around though
Name: Email, dtype: object

In [72]:
#Separating text(input) and target classes
list_sentences_rawdata = Email_Data["Email"].fillna("_na_").values
list_classes = ["Target"]
target = Email_Data[list_classes].values
To_Process=Email_Data[['Email', 'Target']]


In [73]:
#Train and test split with 80:20 ratio
train, test = train_test_split(To_Process, test_size=0.2)

In [74]:
# Define the sequence lengths, max number of words and embedding dimensions
# Sequence length of each sentence. If more, truncate. If less, pad with zeros
MAX_SEQUENCE_LENGTH = 300

# Top 20000 frequently occurring words
MAX_NB_WORDS = 20000

# Get the frequently occurring words
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train.Email)
train_sequences = tokenizer.texts_to_sequences(train.Email)
test_sequences = tokenizer.texts_to_sequences(test.Email)


# dictionary containing words and their index
word_index = tokenizer.word_index
# print(tokenizer.word_index)
# total words in the corpus
print('Found %s unique tokens.' % len(word_index))

Found 8350 unique tokens.


In [75]:
# get only the top frequent words on train
train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [76]:
# get only the top frequent words on test
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [77]:
print(train_data.shape)
print(test_data.shape)

(4457, 300)
(1115, 300)


In [78]:
train_labels = train['Target']
test_labels = test['Target'] 

In [79]:
from sklearn.preprocessing import LabelEncoder
# converts the character array to numeric array. Assigns levels to unique labels.

In [80]:
le = LabelEncoder()
le.fit(train_labels)
train_labels = le.transform(train_labels)
test_labels = le.transform(test_labels)

In [81]:
# changing data types

import numpy as np

labels_train = to_categorical(np.asarray(train_labels))
labels_test = to_categorical(np.asarray(test_labels))
print('Shape of data tensor:', train_data.shape)
print('Shape of label tensor:', labels_train.shape)
print('Shape of label tensor:', labels_test.shape)

Shape of data tensor: (4457, 300)
Shape of label tensor: (4457, 2)
Shape of label tensor: (1115, 2)


In [88]:
EMBEDDING_DIM = 100
print(MAX_SEQUENCE_LENGTH)

300


# CNN

In [83]:
# learning the model 
model = Sequential()
model.add(Embedding(MAX_NB_WORDS,EMBEDDING_DIM,input_length=MAX_SEQUENCE_LENGTH))
model.add(Dropout(0.5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [84]:
model.fit(train_data, labels_train, batch_size=64, epochs=5, validation_data=(test_data, labels_test))


Train on 4457 samples, validate on 1115 samples
Epoch 1/5
4457/4457 [==============================] - 22s 5ms/step - loss: 0.4009 - acc: 0.8378 - val_loss: 0.4620 - val_acc: 0.8457
Epoch 2/5
4457/4457 [==============================] - 20s 4ms/step - loss: 0.1595 - acc: 0.9435 - val_loss: 0.6216 - val_acc: 0.8457
Epoch 3/5
4457/4457 [==============================] - 19s 4ms/step - loss: 0.0778 - acc: 0.9782 - val_loss: 0.3608 - val_acc: 0.8457
Epoch 4/5
4457/4457 [==============================] - 19s 4ms/step - loss: 0.0488 - acc: 0.9886 - val_loss: 0.4370 - val_acc: 0.8619
Epoch 5/5
4457/4457 [==============================] - 18s 4ms/step - loss: 0.0389 - acc: 0.9910 - val_loss: 0.5438 - val_acc: 0.9534


In [85]:
#predictions on test data
predicted=model.predict(test_data)
predicted

array([[0.5295881 , 0.47041187],
       [0.576681  , 0.42331898],
       [0.58025044, 0.41974956],
       ...,
       [0.24285339, 0.75714666],
       [0.5782141 , 0.4217859 ],
       [0.57895905, 0.42104095]], dtype=float32)

In [86]:
#model evaluation
import sklearn
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(labels_test, predicted.round())
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))

precision: [0.94864048 0.99180328]
recall: [0.99893955 0.70348837]


In [87]:
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print("############################")
print(sklearn.metrics.classification_report(labels_test, predicted.round()))

fscore: [0.9731405  0.82312925]
support: [943 172]
############################
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       943
           1       0.99      0.70      0.82       172

   micro avg       0.95      0.95      0.95      1115
   macro avg       0.97      0.85      0.90      1115
weighted avg       0.96      0.95      0.95      1115
 samples avg       0.95      0.95      0.95      1115



# RNN 

In [89]:
#import library
from keras.layers.recurrent import SimpleRNN
#model training
print('Training SIMPLERNN model.')
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH ))
model.add(SimpleRNN(2, input_shape=(None,1)))
model.add(Dense(2,activation='softmax'))

Training SIMPLERNN model.


In [90]:
model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics = ['accuracy']) 

In [91]:
model.fit(train_data, labels_train,batch_size=16,epochs=5,validation_data=(test_data, labels_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 4457 samples, validate on 1115 samples
Epoch 1/5
4457/4457 [==============================] - 29s 6ms/step - loss: 0.4072 - accuracy: 0.9437 - val_loss: 0.2764 - val_accuracy: 0.9722
Epoch 2/5
4457/4457 [==============================] - 28s 6ms/step - loss: 0.1791 - accuracy: 0.9897 - val_loss: 0.1975 - val_accuracy: 0.9713
Epoch 3/5
4457/4457 [==============================] - 27s 6ms/step - loss: 0.0835 - accuracy: 0.9953 - val_loss: 0.1472 - val_accuracy: 0.9623
Epoch 4/5
4457/4457 [==============================] - 28s 6ms/step - loss: 0.0383 - accuracy: 0.9980 - val_loss: 0.1427 - val_accuracy: 0.9489
Epoch 5/5
4457/4457 [==============================] - 28s 6ms/step - loss: 0.0223 - accuracy: 0.9987 - val_loss: 0.1429 - val_accuracy: 0.9489


In [92]:
# prediction on test data
predicted_Srnn=model.predict(test_data)
predicted_Srnn

array([[0.03912038, 0.9608796 ],
       [0.97968644, 0.02031357],
       [0.9459999 , 0.05400007],
       ...,
       [0.11900423, 0.88099575],
       [0.8923687 , 0.1076313 ],
       [0.76538324, 0.2346168 ]], dtype=float32)

In [93]:
#model evaluation
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(labels_test, predicted_Srnn.round())

In [94]:
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print("############################")
print(sklearn.metrics.classification_report(labels_test,predicted_Srnn.round()))

precision: [0.94657258 0.96747967]
recall: [0.99575822 0.69186047]
fscore: [0.97054264 0.80677966]
support: [943 172]
############################
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       943
           1       0.97      0.69      0.81       172

   micro avg       0.95      0.95      0.95      1115
   macro avg       0.96      0.84      0.89      1115
weighted avg       0.95      0.95      0.95      1115
 samples avg       0.95      0.95      0.95      1115



# LSTM model 

In [96]:
#model training
print('Training LSTM model.')
model = Sequential()
model.add(Embedding(MAX_NB_WORDS,EMBEDDING_DIM,input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(output_dim=16, activation='relu', inner_activation='hard_sigmoid',return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

Training LSTM model.


C:\Users\baban\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="relu", return_sequences=True, units=16, recurrent_activation="hard_sigmoid")`
  """


In [97]:
model.add(Flatten())
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics = ['accuracy'])
model.fit(train_data, labels_train,batch_size=16,epochs=5,validation_data=(test_data, labels_test))

Train on 4457 samples, validate on 1115 samples
Epoch 1/5
4457/4457 [==============================] - 62s 14ms/step - loss: 0.1272 - accuracy: 0.9565 - val_loss: 0.2181 - val_accuracy: 0.9623
Epoch 2/5
4457/4457 [==============================] - 64s 14ms/step - loss: 0.0134 - accuracy: 0.9964 - val_loss: 0.0734 - val_accuracy: 0.9767
Epoch 3/5
4457/4457 [==============================] - 63s 14ms/step - loss: 0.0026 - accuracy: 0.9996 - val_loss: 0.0889 - val_accuracy: 0.9785
Epoch 4/5
4457/4457 [==============================] - 59s 13ms/step - loss: 0.0015 - accuracy: 0.9998 - val_loss: 0.0807 - val_accuracy: 0.9821
Epoch 5/5
4457/4457 [==============================] - 61s 14ms/step - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.1095 - val_accuracy: 0.9776


In [98]:
#prediction on text data
predicted_lstm=model.predict(test_data)
predicted_lstm

array([[6.5197310e-05, 9.9993479e-01],
       [9.6274668e-01, 3.7253309e-02],
       [9.9966562e-01, 3.3437929e-04],
       ...,
       [2.1405364e-12, 1.0000000e+00],
       [9.9982184e-01, 1.7818072e-04],
       [9.9991643e-01, 8.3569561e-05]], dtype=float32)

In [101]:
#model evaluation
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(labels_test,
predicted_lstm.round())
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print("############################")

print(sklearn.metrics.classification_report(labels_test,predicted_lstm.round()))

precision: [0.97614108 0.98675497]
recall: [0.99787911 0.86627907]
fscore: [0.9868904  0.92260062]
support: [943 172]
############################
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       943
           1       0.99      0.87      0.92       172

   micro avg       0.98      0.98      0.98      1115
   macro avg       0.98      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115
 samples avg       0.98      0.98      0.98      1115



# Bidirectional LSTM

In [103]:
#model training
print('Training Bidirectional LSTM model.') 
model = Sequential()
model.add(Embedding(MAX_NB_WORDS,EMBEDDING_DIM,input_length=MAX_SEQUENCE_LENGTH))
model.add(Bidirectional(LSTM(16, return_sequences=True,dropout=0.1, recurrent_dropout=0.1)))
model.add(Conv1D(16, kernel_size = 3, padding = "valid",kernel_initializer = "glorot_uniform"))
model.add(GlobalMaxPool1D())
model.add(Dense(50, activation="relu"))
model.add(Dropout(0.1))

Training Bidirectional LSTM model.


In [104]:
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics = ['accuracy'])
model.fit(train_data, labels_train,batch_size=16,epochs=3,validation_data=(test_data, labels_test))

Train on 4457 samples, validate on 1115 samples
Epoch 1/3
4457/4457 [==============================] - 73s 16ms/step - loss: 0.1543 - accuracy: 0.9457 - val_loss: 0.0805 - val_accuracy: 0.9830
Epoch 2/3
4457/4457 [==============================] - 63s 14ms/step - loss: 0.0153 - accuracy: 0.9969 - val_loss: 0.0732 - val_accuracy: 0.9839
Epoch 3/3
4457/4457 [==============================] - 65s 15ms/step - loss: 0.0028 - accuracy: 0.9993 - val_loss: 0.0695 - val_accuracy: 0.9848


In [105]:
# prediction on test data
predicted_blstm=model.predict(test_data)
predicted_blstm

array([[5.5050768e-04, 9.9944943e-01],
       [9.2666931e-02, 9.0733308e-01],
       [9.9864823e-01, 1.3517353e-03],
       ...,
       [2.4379567e-06, 9.9999762e-01],
       [9.9972814e-01, 2.7180091e-04],
       [9.9940467e-01, 5.9538323e-04]], dtype=float32)

In [106]:
#model evaluation
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(labels_test,predicted_blstm.round())
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print("############################")

precision: [0.98430962 0.98742138]
recall: [0.99787911 0.9127907 ]
fscore: [0.99104792 0.94864048]
support: [943 172]
############################


In [107]:
print(sklearn.metrics.classification_report(labels_test,predicted_blstm.round()))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       943
           1       0.99      0.91      0.95       172

   micro avg       0.98      0.98      0.98      1115
   macro avg       0.99      0.96      0.97      1115
weighted avg       0.98      0.98      0.98      1115
 samples avg       0.98      0.98      0.98      1115

